In [1]:
!pip install tweepy

In [2]:
!pip install openpyxl

In [1]:
# импорт используемых библиотек
import time
import tweepy
import openpyxl

In [4]:
# основная функция парсинга
def get_tweets(username, hashtags):  # параметры (массивы) получаем из main()
    client = tweepy.Client(
        "AAAAAAAAAAAAAAAAAAAAAE5QcAEAAAAAzWLMOWdWgSupTjB2ja6Yai3ny7M%3DeyGhScn8Mg13AotaHE0wUjhXzSPSlGPdpfFfsiYRiE0XpSBSHy",
        wait_on_rate_limit=True)  # создали объект для работы с АПИ

    id_ = client.get_user(username=username, user_fields='id').data['id']  # username превращаем в id (для работы с АПИ)
    tweets = client.get_users_tweets(id=id_, tweet_fields='public_metrics,created_at', max_results=100)  # получение доступа к твитам

    # работа с экселем - создание файла
    book_name = 'XRP2.xlsx'
    try:
        book = openpyxl.load_workbook(book_name)
    except:
        book = openpyxl.Workbook()

    sheet = book.active
    sheet['A1'].value = 'Text'
    sheet['B1'].value = 'Date'
    sheet['C1'].value = 'Likes'
    sheet['D1'].value = 'Comments'
    sheet['E1'].value = 'Retweets'
    sheet['F1'].value = 'Views'
    book.save(book_name)

    # парсер твитов
    i = 0
    while True:  # цикл, в котором пробегаемся по каждой порции (100 штук) твитов (легальный обход некоторых ограничений твиттера)
        i += 100
        if tweets.data is not None:
            print(i)  # вывод кол-ва проверенных твитов
            for tweet in tweets.data:  # пробегаемся по полученным объектам твитов
                to_excel = []  # массив - одна строка экселя

                # text
                is_hashtag_in_text = False  # для проверки на наличе тэга в твите
                try:
                    text = tweet['text'].replace('\n', ' ')
                except:  # если сбой со стороны АПИ - смотрим следующий твит, тк остальная инфа не важна без текста твита
                    continue

                for tag in hashtags:  # проверка на наличие хэштэгов в текста
                    if tag.lower() in text or tag.upper() in text:
                        is_hashtag_in_text = True
                        break

                if is_hashtag_in_text:  # если хэштэг нашелся в твите, работаем дальше
                    to_excel.append(text)
                else:  # нет хэштэга - пропускаем твит
                    continue


                # далее идет добавление каждого требуемого параметра в "массив строки экселя" - работа с АПИ
                # date
                try:
                    created_at = tweet['created_at'].strftime("%d-%m-%Y %H:%M:%S")
                except:  # если сбой со стороны АПИ - смотрим следующий твит, тк остальная инфа не важна без времени твита
                    continue
                to_excel.append(created_at)


                # likes
                try:
                    likes = tweet['public_metrics']['like_count']
                except:
                    likes = 0
                to_excel.append(likes)


                # replies
                try:
                    replies = tweet['public_metrics']['reply_count']
                except:
                    replies = 0
                to_excel.append(replies)


                # retweets
                try:
                    retweets = tweet['public_metrics']['retweet_count']
                except:
                    retweets = 0
                to_excel.append(retweets)


                # views
                try:
                    views = tweet['public_metrics']['impression_count']
                except:
                    views = 0
                to_excel.append(views)

                sheet.append(to_excel)  # запись массива добавленным выше параметрами в строку экселя, далее переходим к новому твиту в цикле

            book.save(book_name)

            try:
                next_token_user = tweets.meta['next_token']  # получение токена для доступа к следующим твитам
                tweets = client.get_users_tweets(id=id_, pagination_token=next_token_user, max_results=100)  # переход к новой порции твитов
            except Exception as E:
                E = str(E)
                if "'next_token'" == E:  # если ошибка в том, что нет токенов больше, значит твиты кончились - заверешение работы
                    book.save(book_name)
                    book.close()
                    return

                print(f'error: {E}', tweets)  # если другая ошибка - вывод в консоль, для дальнейшего разбора
                book.save(book_name)
                time.sleep(1)
                pass

        if tweets.data is None:  # если твиты кончились - завершение работы
            break

    book.close()  # конец работы с экселем



# здесь происходит запуск функции парсинга
def main():
    # 1. логин отсюда https://twitter.com/opensea__nfts (подстрока после последнего "/" в ссылке на профиль)
    # 2. есть два массива usernames и hashtags - заполняете их для добавления аккаунтов и тэгов для парсинга
    # 2.1. Как добавить аккаунт PKlaphan:
    #           был массив usernames = ['opensea__nfts'], после добавления стал usernames = ['opensea__nfts', 'PKlaphan']
    #           то есть просто через запятую так же дописываете элементы в него
    # 2.2. аналогично для хэштэгов (не забудьте про пробел в конце слова)

    usernames = ['ZAYKCharts', 'LiskHQ', 'qtum', 'EOSIO', 'loopringorg', 'iota', 'AltcoinDailyio', 'IncomeSharks', 'whale_alert','santimentfeed','coingecko','airdropinspect','Airdroppro_','punk6529','CryptoYoda1338','SteemNetwork']
    hashtags = ['XRP ', 'Ripple ']  # регистр не важен. если хотите поиск по отдельному слову (а не просто сочетание букв) в конце слова пробел ставьте,
    # hashtag = ['']  # откомментируйте эту строку (уберите первую решетку в строке), если не хотите делать поиск по слову
 #'steemit', 'Neo_Blockchain', 'Ripple', 'Ripple_XRP1', 'XRP_community', 'XRPcryptowolf', 'BTCTN', 'BTC_Archive', 'Bitcoin', 'rektcapital', 'aantonop', 'woonomic', 'monero', 'litecoin', 'SatoshiLite', 'LiteCoinNews', 'trondao', 'TronixTrx', 'VitalikButerin', 'ethereum', 'ethereumJoseph',	'eth_classic', 'ETC', 'ShibariumNet', 'ShibainuCoin', 'Shibtoken', 'dogecoin', 'ConsciousDoge', 'DogecoinRide', 'elonmusk', 'solana', 'vergecurrency', 'adahealth', 'Cardano', '1inch', 'cryptofreemium', 'PancakeSwap', 'CryptoBillu', 'clifton_ideas', 'StellarOrg', 'Uniswap', 'Nebraskangooner', 'gmgiray', 'Polkadot', 'eliz883', 'Altcoinbuzzio', 'gavofyork', 'BoredApeYC', 'RyanWilliams629', 'RunOnFlux', 'quant_network', 'ZcashFoundation', 'zcash', 'BullsCoin', 'ShrayTV', 'missedamillion', 'TheCryptoLark', 
    for username in usernames:  # пробегаемся по всем логинам из заданного массива
        print(f'\nРабота с твитами {username}')
        get_tweets(username, hashtags)  # вызов функции парсинга, передаем массив ников и хэштэгов для дальнейшей работы


# вызов общей функции
if __name__ == '__main__':
    main()



Работа с твитами ZAYKCharts
100
200
300
400
500
600
700
800
900
1000
1100

Работа с твитами LiskHQ
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300

Работа с твитами qtum
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900

Работа с твитами EOSIO
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300

Работа с твитами loopringorg
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300

Работа с твитами iota
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300

Работа с твитами AltcoinDailyio
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2